# **Amazon Lookout for Equipment** - Getting started
*Part 7 - Cleanup*

## Initialization
---
This repository is structured as follow:

```sh
. lookout-equipment-demo
|
├── data/
|   ├── interim                          # Temporary intermediate data are stored here
|   ├── processed                        # Finalized datasets are usually stored here
|   |                                    # before they are sent to S3 to allow the
|   |                                    # service to reach them
|   └── raw                              # Immutable original data are stored here
|
├── getting_started/
|   ├── 1_data_preparation.ipynb
|   ├── 2_dataset_creation.ipynb
|   ├── 3_model_training.ipynb
|   ├── 4_model_evaluation.ipynb
|   ├── 5_inference_scheduling.ipynb
|   └── 6_cleanup.ipynb                  <<< THIS NOTEBOOK <<<
|
└── utils/
    └── lookout_equipment_utils.py
```

### Notebook configuration update

In [ ]:
!pip install --quiet --upgrade sagemaker tqdm

### Imports

In [ ]:
import boto3
import config
import datetime
import sagemaker
import sys
import time

# Helper functions for managing Lookout for Equipment API calls:
sys.path.append('../utils')
import lookout_equipment_utils as lookout

In [ ]:
ROLE_ARN                 = sagemaker.get_execution_role()
REGION_NAME              = boto3.session.Session().region_name
BUCKET                   = config.BUCKET
PREFIX_TRAINING          = config.PREFIX_TRAINING
PREFIX_LABEL             = config.PREFIX_LABEL
PREFIX_INFERENCE         = config.PREFIX_INFERENCE
DATASET_NAME             = config.DATASET_NAME
MODEL_NAME               = config.MODEL_NAME
INFERENCE_SCHEDULER_NAME = config.INFERENCE_SCHEDULER_NAME

In [ ]:
lookout_client = lookout.get_client(region_name=REGION_NAME)

## Deleting resources
---
### Deleting inference scheduler
Using the [**DeleteInferenceScheduler**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DeleteInferenceScheduler.html) API to delete existing scheduler:

In [ ]:
# Stopping the scheduler in case it's running:
try:
    print('Stopping the scheduler...')
    scheduler = lookout.LookoutEquipmentScheduler(
        scheduler_name=INFERENCE_SCHEDULER_NAME,
        model_name=MODEL_NAME,
        region_name=REGION_NAME
    )
    scheduler.stop()
    scheduler.delete()
    
except Exception as e:
    error_code = e.response['Error']['Code']
    if (error_code == 'ResourceNotFoundException'):
        print('  > Scheduler not found, nothing to do')

### Deleting the trained models
Using the [**DeleteModel**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DeleteModel.html) API to remove the model trained in this tutorial:

In [ ]:
for model in lookout.list_models_for_datasets(model_name_prefix=MODEL_NAME):
    print(f'Deleting model {model}...')
    
    try:
        lookout_client.delete_model(ModelName=MODEL_NAME)
        print(f'Model "{MODEL_NAME}" is deleted successfully.')

    except Exception as e:
        error_code = e.response['Error']['Code']
        # If the dataset is used by existing models and we asked a
        # forced delete, we also delete the associated models before
        # trying again the dataset deletion:
        if (error_code == 'ConflictException'):
            print(('Model is currently being used (a training might be in '
                   'progress. Wait for the process to be completed and '
                   'retry.'))

### Deleting the dataset
Using the [**DeleteDataset**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DeleteDataset.html) API to remove the dataset:

In [ ]:
# Let's try to delete this dataset:
try:
    lookout_client.delete_dataset(DatasetName=DATASET_NAME)
    print(f'Dataset "{DATASET_NAME}" is deleted successfully.')

except Exception as e:
    error_code = e.response['Error']['Code']
    if (error_code == 'ConflictException'):
        print(('Dataset is used by at least a model, delete the '
               'associated model(s) before deleting this dataset.'))

### Cleaning the S3 bucket
Uncomment and run the following cell to clean the S3 bucket from the prefixes used throughout this tutorial for training data, label data and inference data. You can stop here if you would like to keep the data generated for further experimentation:

In [ ]:
# !aws s3 rm s3://$BUCKET/$PREFIX_INFERENCE --recursive
# !aws s3 rm s3://$BUCKET/$PREFIX_TRAINING --recursive
# !aws s3 rm s3://$BUCKET/$PREFIX_LABEL --recursive

## Conclusion
---

Use this notebook to cleanup all the ressources created while running this series of tutorials.